In [1]:
import ase
import ase.io
import jax
import jax.numpy as jnp
import jraph
import ml_collections
import matscipy
import numpy as np
import pandas as pd
import sys

sys.path.append('..')
from analyses import analysis
from symphony import train
from symphony.data import input_pipeline_tf

In [14]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [15]:
workdir = '/data/NFS/potato/songk/spherical-harmonic-net/workdirs/silica_shuffled_matscipy_jan24/e3schnet_and_nequip/nn/max_targets_4'
rng = jax.random.PRNGKey(0)

In [16]:
config, best_state, best_eval_state, metrics_for_best_state = analysis.load_from_workdir(workdir)
config = ml_collections.FrozenConfigDict(config)
datasets = input_pipeline_tf.get_datasets(rng, config)

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 301582570443555955
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 301582570443555955
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text

In [4]:
metrics_for_best_state

NameError: name 'metrics_for_best_state' is not defined

In [5]:
frag_batch = next(datasets['test'].as_numpy_iterator())
unbatched_frags = jraph.unbatch(frag_batch)

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 13455037253616018397


In [6]:
frag = unbatched_frags[2]

In [7]:
step_rng, rng = jax.random.split(rng)
step_rngs = jax.random.split(step_rng, jax.local_device_count())
preds = train.get_predictions(best_eval_state, frag, rng)

In [16]:
frag_ase = ase.Atoms(
    positions = frag.nodes.positions,
    numbers = np.vectorize(lambda x: 14 if x else 8)(frag.nodes.species),
    cell = frag.globals.cell[0],
    pbc = np.array([1, 1, 1]),
)
# ase.io.write('frag.cif', frag_ase)

In [9]:
frag.nodes.species

Array([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1], dtype=int32)

In [10]:
frag.globals.stop, preds.globals.stop

(Array([False], dtype=bool), Array([False], dtype=bool))

In [11]:
frag.nodes.focus_and_target_species_probs

Array([[0.16666667, 0.        ],
       [0.16666667, 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.16666667, 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.

In [12]:
jnp.where(preds.nodes.focus_and_target_species_probs > 1e-4, preds.nodes.focus_and_target_species_probs, 0
          )

Array([[0.03672149, 0.        ],
       [0.06271943, 0.        ],
       [0.04040426, 0.        ],
       [0.04044089, 0.        ],
       [0.        , 0.0083559 ],
       [0.        , 0.        ],
       [0.        , 0.00835029],
       [0.        , 0.        ],
       [0.        , 0.00817594],
       [0.        , 0.        ],
       [0.        , 0.00528534],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.01566604, 0.        ],
       [0.15435658, 0.        ],
       [0.        , 0.        ],
       [0.00270433, 0.        ],
       [0.07377874, 0.        ],
       [0.        , 0.        ],
       [0.12360542, 0.        ],
       [0.        , 0.0083559 ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.0109337 ],
       [0.        , 0.        ],
       [0.

In [27]:
preds.globals.position_probs

SphericalSignal(shape=(1, 64, 90, 179), res_beta=90, res_alpha=179, quadrature=gausslegendre, p_val=1, p_arg=-1)
[[[[6.2527385e-04 6.2495726e-04 6.2463735e-04 ... 6.2619464e-04
    6.2589196e-04 6.2558753e-04]
   [6.2910124e-04 6.2823074e-04 6.2734587e-04 ... 6.3162314e-04
    6.3079904e-04 6.2996102e-04]
   [6.3355546e-04 6.3195633e-04 6.3033239e-04 ... 6.3814461e-04
    6.3665112e-04 6.3512230e-04]
   ...
   [1.5002221e-06 1.4944644e-06 1.4883542e-06 ... 1.5153288e-06
    1.5106567e-06 1.5056227e-06]
   [1.4512889e-06 1.4476504e-06 1.4438946e-06 ... 1.4614500e-06
    1.4581894e-06 1.4548031e-06]
   [1.4274348e-06 1.4258403e-06 1.4242394e-06 ... 1.4321499e-06
    1.4305912e-06 1.4290175e-06]]

  [[1.3411281e-03 1.3389493e-03 1.3367612e-03 ... 1.3475294e-03
    1.3454158e-03 1.3432864e-03]
   [1.3555518e-03 1.3503365e-03 1.3450797e-03 ... 1.3708989e-03
    1.3658409e-03 1.3607288e-03]
   [1.3667569e-03 1.3583003e-03 1.3497930e-03 ... 1.3916803e-03
    1.3834587e-03 1.3751494e-03]
   ..

In [21]:
frag_ase.numbers

array([14, 14, 14, 14,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
       14, 14, 14, 14, 14, 14, 14, 14,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8, 14, 14, 14, 14])